In [1]:
from vpython import *
scene=canvas(width=800,height=800,background=color.black) # Create a canvas for 3D graphics, align to the left
g=graph(width=scene.width,height=scene.height, background=color.gray(0.6)) # Create a graph for plotting, align to the right
gc=gcurve(graph=g,color=color.yellow) 
gc2=gcurve(graph=g,color=color.red) 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
g=9.8
pi=3.1415926
size=0.5
mass=1
Natoms=300
L=5
dt=0.02
tofbig=2 #times of radius
mtofbig=1 #times of mass
t=0
tp=0
R=2 #amplitude
T=1 #period


container=box(length = 2*L , height = 2*L, width = 2*L,  opacity=0.2, color = color.yellow)
container.pos=vector(0,0,0)
big_nut=sphere(radius=tofbig*size,color=color.cyan)
big_nut.pos=vector(0,-3,0)
big_nut.v=vector(0,0,0)
mbig_nut=mtofbig*mass
atoms=[]

for i in range(Natoms):              #設定此N個原子的初始條件，其位置隨機在盒子內，速度的方向也隨機，其中一個原子要畫軌跡
    position = vector(-0.9*L+1.8*L*random(),-0.9*L+1.8*L*random(),-0.9*L+1.8*L*random())
    atom = sphere(pos=position, radius = size, color=color.red)
    matom=mass
    atom.v = vector(random(),random(),random())
    atoms.append(atom)
    
def vcollision(b1,b2,m1,m2):      #彈性碰撞公式
    v1_par= (b1.pos-b2.pos)  * dot (b1.v, b1.pos-b2.pos) / mag(b1.pos-b2.pos)**2
    v1=b1.v-v1_par
    v1_par_cpy=v1_par
    v2_par= (b2.pos-b1.pos)  * dot (b2.v, b2.pos-b1.pos) / mag(b2.pos-b1.pos)**2
    v2=b2.v-v2_par
    v2_par_cpy=v2_par
    v1_par=((m1-m2)/(m1+m2))*v1_par_cpy
    v1_par+=((2*m2)/(m1+m2))*v2_par_cpy
    v2_par=((m2-m1)/(m2+m1))*v2_par_cpy
    v2_par+=((2*m1)/(m2+m1))*v1_par_cpy
    v1=v1+v1_par
    v2=v2+v2_par
    return v1, v2
    
def abs(k):
    if k<0:
        return -k
    else:
        return k
        
def mag(a):
    sum=a.x**2+a.y**2+a.z**2
    return sum**(1/2)
     

##ev = scene.waitfor('click keydown')

while True:
    rate(50)
    sum=0
    v_container=0
    
    if tp<T:    #approximate box_shaking as SHM
        v_container=2*pi*R*(cos(2*3.1416*t/T))/T
        container.pos += v_container*vector(1,0,0)*dt
    else:
        v_container=0
    
##    if ev.event == 'click':
##        tp=0
    
    big_nut.v += g*vector(0,-1,0) * dt
    big_nut.pos += big_nut.v * dt
    for atom in atoms:
        atom.v += g*vector(0,-1,0) * dt
        atom.pos += atom.v * dt  #位移 = 速度*時間
    
    for i in range(Natoms-1): #此四行檢查任2個原子間，是否發生碰撞，若是則以彈性碰撞公式設定碰撞後的速度
        sum += atoms[i].pos.y
        if mag(atoms[i].pos - big_nut.pos) <= (1+tofbig)*size and dot(atoms[i].pos-big_nut.pos, atoms[i].v-big_nut.v) < 0 :
                atoms[i].v, big_nut.v = vcollision(atoms[i], big_nut,matom,mbig_nut)
                atoms[i].v= atoms[i].v*1
                big_nut.v= big_nut.v*1
        
        for j in range(i,Natoms):    
            if mag(atoms[i].pos - atoms[j].pos) <= 2*size and dot(atoms[i].pos-atoms[j].pos, atoms[i].v-atoms[j].v) < 0 :
                atoms[i].v, atoms[j].v = vcollision(atoms[i], atoms[j],matom,matom)
                atoms[i].v =atoms[i].v*1
                atoms[j].v =atoms[j].v*1
    
    for atom in atoms:        #簡查所有原子是否撞牆  
        if atom.pos.x + size >= L+container.pos.x or (atom.pos.x-size)<= -L+container.pos.x:     #判斷是否撞牆
           ## if atom.pos.x*atom.v.x >= 0
            atom.v.x = - atom.v.x+v_container        #若撞牆，該原子在該維度方向的速度設為相反
        if (abs(atom.pos.y) + size) >= L and atom.pos.y*atom.v.y > 0 :     #判斷是否撞牆
                atom.v.y = - atom.v.y         #若撞牆，該原子在該維度方向的速度設為相反
        if abs(atom.pos.z) + size >= L and atom.pos.z*atom.v.z > 0 :     #判斷是否撞牆
                atom.v.z = - atom.v.z         #若撞牆，該原子在該維度方向的速度設為相反
    if big_nut.pos.x + tofbig >= L+container.pos.x or big_nut.pos.x - tofbig <= -L+container.pos.x:     #判斷是否撞牆
        if big_nut.pos.x *big_nut.v.x >0:
            big_nut.v.x = - big_nut.v.x+v_container        #若撞牆，該原子在該維度方向的速度設為相反
    if abs(big_nut.pos.y) + tofbig*size >= L and big_nut.pos.y*big_nut.v.y > 0 :     #判斷是否撞牆
            big_nut.v.y = - big_nut.v.y         #若撞牆，該原子在該維度方向的速度設為相反
    if abs(big_nut.pos.z) + tofbig*size >= L and big_nut.pos.z*big_nut.v.z > 0 :     #判斷是否撞牆
            big_nut.v.z = - big_nut.v.z         #若撞牆，該原子在該維度方向的速度設為相反
    gc.plot([t,big_nut.pos.y])
    gc2.plot([t,sum/Natoms])
    print (tp)
    t += dt
    tp += dt